In [125]:
import librosa
import pyaudio
import os,glob,pickle
import soundfile
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score 
import numpy as np

In [126]:
def extract_feature(filename,mfcc,chroma,mel):
    with soundfile.SoundFile(filename) as f:
        X=f.read(dtype='float32')
        print(X.shape)
        sample_rate=f.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
            result=np.hstack((result,mfcc))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
            result=np.hstack((result,chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X,sr=sample_rate).T,axis=0)
            result=np.hstack((result,mel)) 
    return result       




In [127]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [128]:
def load_data(test_size=0.2):
    x=[]
    y=[]
    for file in glob.glob('F:\\download\\ravdess\\Actor_*\\*.wav'):
        print('processing..........')
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split('-')[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
        x.append(feature)
        y.append(emotion)    
    return train_test_split(np.array(x),y,test_size=test_size,random_state=42)    

In [129]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

..........
(60327,)
processing..........
(60327,)
processing..........
(58725,)
processing..........
(63530,)
processing..........
(60327,)
processing..........
(59793,)
processing..........
(60861,)
processing..........
(59259,)
processing..........
(59793,)
processing..........
(56056,)
processing..........
(59793,)
processing..........
(67267,)
processing..........
(66199,)
processing..........
(62463,)
processing..........
(63530,)
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
processing..........
(54455,)
processing..........
(56056,)
processing..........
(53920,)
processing..........
(51251,)
processing..........
(59793,)
processing..........
(59259,)
processing..........
(55522,)
processing....

In [132]:
print((x_train.shape, x_test.shape))

((576, 141), (192, 141))


In [133]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 141


In [134]:
model=MLPClassifier(alpha=0.01, batch_size=32, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=900)

In [135]:
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=32, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=900,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [136]:
y_pred=model.predict(x_test)

In [137]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 54.17%


In [138]:
pickle.dump(model, open("mlp_classifier.model", "wb"))

In [131]:
file='Recording1.wav'
feature=extract_feature(file,mfcc=True,chroma=True,mel=True)

(162816, 2)


ParameterError: Invalid shape for monophonic audio: ndim=2, shape=(162816, 2)

In [120]:
model.predict(feature.reshape(1,-1))

array(['calm'], dtype='<U7')